In [103]:
import numpy as np
import pandas as pd
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/kakr-4th-competition/train.csv
/kaggle/input/kakr-4th-competition/test.csv
/kaggle/input/kakr-4th-competition/sample_submission.csv


In [104]:
train_df = pd.read_csv('../input/kakr-4th-competition/train.csv')
test_df = pd.read_csv('../input/kakr-4th-competition/test.csv')

# 1. Data Labeling

In [105]:
train_df.drop(['id'], axis=1, inplace=True)
test_df.drop(['id'], axis=1, inplace=True)

In [106]:
y = train_df['income'] != '<=50K'
X = train_df.drop(['income'], axis=1)

In [107]:
# 라벨 인코더 생성
LE_encoder = OrdinalEncoder(list(X.columns))

# train, test 데이터에 인코딩 적용
X = LE_encoder.fit_transform(X, y)
test = LE_encoder.transform(test)

In [108]:
X['income'] = y

In [109]:
X.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,40,1,168538,1,9,1,1,1,1,1,0,0,60,1,True
1,17,1,101626,2,5,2,2,2,1,1,0,0,20,1,False
2,18,1,353358,3,10,2,3,2,1,1,0,0,16,1,False
3,21,1,151158,3,10,2,4,2,1,2,0,0,25,1,False
4,24,1,122234,3,10,2,5,3,2,2,0,0,20,2,False


In [110]:
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

In [111]:
y_train = X_train.income
X_train = X_train.drop(['income'], axis=1)
y_val = X_val.income
X_val = X_val.drop(['income'], axis=1)

In [112]:
#batch_size = 5
#train_ds = df_to_dataset(X_train, batch_size=batch_size)
#val_ds = df_to_dataset(X_val, shuffle=False, batch_size=batch_size)
#test_ds = df_to_dataset(test, )

# 2. Neural Network with TensorFlow

In [113]:
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [114]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [119]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/20
652/652 [==============================] - 1s 2ms/step - loss: 103.6445 - accuracy: 0.6829
Epoch 2/20
652/652 [==============================] - 1s 2ms/step - loss: 93.3850 - accuracy: 0.6841
Epoch 3/20
652/652 [==============================] - 1s 2ms/step - loss: 79.5055 - accuracy: 0.6852
Epoch 4/20
652/652 [==============================] - 1s 2ms/step - loss: 58.3462 - accuracy: 0.6849
Epoch 5/20
652/652 [==============================] - 1s 2ms/step - loss: 42.0134 - accuracy: 0.6899
Epoch 6/20
652/652 [==============================] - 1s 2ms/step - loss: 42.0353 - accuracy: 0.6792
Epoch 7/20
652/652 [==============================] - 1s 2ms/step - loss: 28.3144 - accuracy: 0.6852
Epoch 8/20
652/652 [==============================] - 1s 2ms/step - loss: 24.3157 - accuracy: 0.6830
Epoch 9/20
652/652 [==============================] - 1s 2ms/step - loss: 19.7362 - accuracy: 0.6840
Epoch 10/20
652/652 [==============================] - 1s 2ms/step - loss: 15.2028 - accur

In [120]:
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

163/163 - 0s - loss: 2.8609 - accuracy: 0.2393


In [121]:
print(val_loss)
print(val_acc)

2.860893964767456
0.2393474131822586
